In [ ]:
!unzip "/content/archive (7).zip" -d /content/dataset

Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset/Human Faces Dataset/Real Images/000001.jpg  
  inflating: /content/dataset/Human Faces Dataset/Real Images/000002.jpg  
  inflating: /content/dataset/Human Faces Dataset/Real Images/000003.jpg  
  inflating: /content/dataset/Human Faces Dataset/Real Images/000004.jpg  
  inflating: /content/dataset/Human Faces Dataset/Real Images/000005.jpg  
  inflating: /content/dataset/Human Faces Dataset/Real Images/000006.jpg  
  inflating: /content/dataset/Human Faces Dataset/Real Images/000007.jpg  
  inflating: /content/dataset/Human Faces Dataset/Real Images/000008.jpg  
  inflating: /content/dataset/Human Faces Dataset/Real Images/000009.jpg  
  inflating: /content/dataset/Human Faces Dataset/Real Images/000010.jpg  
  inflating: /content/dataset/Human Faces Dataset/Real Images/000011.jpg  
  inflating: /content/dataset/Human Faces Dataset/Real Images/000012.jpg  
  inflating: /content/dataset/Human Faces Dataset

In [ ]:
import torch
import numpy as np
from datasets import load_dataset
from transformers import (
    ViTImageProcessor,
    ViTForImageClassification,
    TrainingArguments,
    Trainer
)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Using device: cuda


In [ ]:
DATASET_DIR = "/content/dataset/Human Faces Dataset"
MODEL_NAME = "google/vit-base-patch16-224"
NUM_LABELS = 2
EPOCHS = 3

In [ ]:
dataset = load_dataset("imagefolder", data_dir=DATASET_DIR)

dataset = dataset["train"].train_test_split(
    test_size=0.15,
    seed=42
)

print("Columns:", dataset["train"].column_names)
print("Labels:", dataset["train"].features["label"].names)

Resolving data files:   0%|          | 0/9630 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Columns: ['image', 'label']
Labels: ['AI-Generated Images', 'Real Images']


In [ ]:
processor = ViTImageProcessor.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

In [ ]:
def preprocess(batch):
    images = [img.convert("RGB") for img in batch["image"]]
    inputs = processor(images, return_tensors="pt")
    batch["pixel_values"] = inputs["pixel_values"]
    return batch

dataset = dataset.map(
    preprocess,
    batched=True,
    remove_columns=["image"]
)

Map:   0%|          | 0/8185 [00:00<?, ? examples/s]

Map:   0%|          | 0/1445 [00:00<?, ? examples/s]

In [ ]:
model = ViTForImageClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS,
    ignore_mismatched_sizes=True
).to(device)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/200 [00:00<?, ?it/s]

ViTForImageClassification LOAD REPORT from: google/vit-base-patch16-224
Key               | Status   |                                                                                        
------------------+----------+----------------------------------------------------------------------------------------
classifier.weight | MISMATCH | Reinit due to size mismatch ckpt: torch.Size([1000, 768]) vs model:torch.Size([2, 768])
classifier.bias   | MISMATCH | Reinit due to size mismatch ckpt: torch.Size([1000]) vs model:torch.Size([2])          

Notes:
- MISMATCH	:ckpt weights were loaded, but they did not match the original empty weight shapes.


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    acc = (preds == labels).mean()
    return {"accuracy": acc}

In [ ]:
args = TrainingArguments(
    output_dir="./ai_face_detector",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=1,   # IMPORTANT
    logging_steps=50,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics
)

for epoch in range(3):
    print(f"\n===== Epoch {epoch+1}/3 =====")
    trainer.train()
    metrics = trainer.evaluate()
    print("Validation accuracy:", metrics["eval_accuracy"])


===== Epoch 1/3 =====


Step,Training Loss
50,0.000004
100,0.000000
150,0.000000
200,0.000000
250,0.000000
300,0.000000
350,0.000000
400,0.000000
450,0.000000
500,0.000000


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Validation accuracy: 1.0

===== Epoch 2/3 =====


...

Validation accuracy: 1.0

===== Epoch 3/3 =====


Step,Training Loss
50,0.000000
100,0.000000
150,0.000000
200,0.000000
250,0.000000
300,0.000000
350,0.000000
400,0.000000
450,0.000000
500,0.000000


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Validation accuracy: 1.0


In [ ]:
pip install timm facenet-pytorch albumentations opencv-python tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-python-headless to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 135.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 83

In [ ]:
import os
import cv2
import torch
from torch.utils.data import Dataset
from facenet_pytorch import MTCNN
import albumentations as A
from albumentations.pytorch import ToTensorV2

class AIFaceDataset(Dataset):
    def __init__(self, root_dir, label, transform=None):
        self.root_dir = root_dir
        self.label = label
        self.transform = transform
        self.images = os.listdir(root_dir)

        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.mtcnn = MTCNN(image_size=224, margin=20, device=self.device)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.images[idx])

        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        face = self.mtcnn(img)

        # fallback if face not detected
        if face is None:
            face = cv2.resize(img, (224, 224))
            face = torch.tensor(face).permute(2, 0, 1) / 255.0

        if self.transform:
            face = face.permute(1, 2, 0).numpy()
            face = self.transform(image=face)["image"]

        return face, torch.tensor(self.label, dtype=torch.float32)


In [ ]:
train_tfms = A.Compose([
    A.Resize(224,224),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.4),
    A.ImageCompression(quality_lower=30, quality_upper=100, p=0.5),
    A.GaussianBlur(p=0.2),
    A.Normalize(),
    ToTensorV2()
])

val_tfms = A.Compose([
    A.Resize(224,224),
    A.Normalize(),
    ToTensorV2()
])


/tmp/ipython-input-3923255905.py:5: UserWarning: Argument(s) 'quality_lower, quality_upper' are not valid for transform ImageCompression
  A.ImageCompression(quality_lower=30, quality_upper=100, p=0.5),


In [ ]:
from torch.utils.data import DataLoader, ConcatDataset

AI_DIR   = "/content/dataset/Human Faces Dataset/AI-Generated Images"
REAL_DIR = "/content/dataset/Human Faces Dataset/Real Images"

real_ds = AIFaceDataset(REAL_DIR, label=0, transform=train_tfms)
fake_ds = AIFaceDataset(AI_DIR, label=1, transform=train_tfms)

dataset = ConcatDataset([real_ds, fake_ds])

loader = DataLoader(
    dataset,
    batch_size=16,
    shuffle=True,
    num_workers=0,      # 🔥 FIX: MUST be 0
    pin_memory=True
)



In [ ]:
import timm
import torch.nn as nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = timm.create_model(
    "vit_base_patch16_224",
    pretrained=True,
    num_classes=1
)

model.to(device)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False

In [ ]:
criterion = nn.BCEWithLogitsLoss()

optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=3e-5,
    weight_decay=1e-4
)

In [ ]:
from tqdm import tqdm

EPOCHS = 10

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0

    for imgs, labels in tqdm(loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        imgs = imgs.to(device)
        labels = labels.unsqueeze(1).to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1} Loss: {running_loss/len(loader):.4f}")

Epoch 1/10: 100%|██████████| 602/602 [10:10<00:00,  1.01s/it]


Epoch 1 Loss: 0.7079


Epoch 2/10: 100%|██████████| 602/602 [09:48<00:00,  1.02it/s]


Epoch 2 Loss: 0.6987


Epoch 3/10: 100%|██████████| 602/602 [09:53<00:00,  1.01it/s]


...

Epoch 4 Loss: 0.6972


Epoch 5/10: 100%|██████████| 602/602 [09:47<00:00,  1.02it/s]


Epoch 5 Loss: 0.6964


Epoch 6/10:  89%|████████▉ | 537/602 [09:17<01:07,  1.04s/it]


KeyboardInterrupt: 

In [ ]:
hi hello

In [ ]:
import os
import cv2
import torch
from torch.utils.data import Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2

class AIFullImageDataset(Dataset):
    def __init__(self, root_dir, label, transform=None):
        self.root_dir = root_dir
        self.label = label
        self.transform = transform
        self.images = os.listdir(root_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.images[idx])

        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if self.transform:
            img = self.transform(image=img)["image"]

        return img, torch.tensor(self.label, dtype=torch.float32)


In [ ]:
train_tfms = A.Compose([
    A.Resize(224,224),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.4),
    A.ImageCompression(quality_lower=40, quality_upper=100, p=0.5),
    A.GaussianBlur(p=0.2),
    A.Normalize(),
    ToTensorV2()
])

val_tfms = A.Compose([
    A.Resize(224,224),
    A.Normalize(),
    ToTensorV2()
])


/tmp/ipython-input-1006791405.py:5: UserWarning: Argument(s) 'quality_lower, quality_upper' are not valid for transform ImageCompression
  A.ImageCompression(quality_lower=40, quality_upper=100, p=0.5),


In [ ]:
from torch.utils.data import DataLoader, ConcatDataset

AI_DIR   = "/content/dataset/Human Faces Dataset/AI-Generated Images"
REAL_DIR = "/content/dataset/Human Faces Dataset/Real Images"

real_ds = AIFullImageDataset(REAL_DIR, 0, train_tfms)
fake_ds = AIFullImageDataset(AI_DIR, 1, train_tfms)

dataset = ConcatDataset([real_ds, fake_ds])

loader = DataLoader(
    dataset,
    batch_size=32,
    shuffle=True,
    num_workers=0
)


In [ ]:
import timm
import torch.nn as nn

device = "cuda" if torch.cuda.is_available() else "cpu"

model = timm.create_model(
    "vit_base_patch16_224",
    pretrained=True,
    num_classes=1
).to(device)


In [ ]:
for param in model.parameters():
    param.requires_grad = False

model.head.requires_grad = True

optimizer = torch.optim.AdamW(
    model.head.parameters(),
    lr=1e-3
)
criterion = nn.BCEWithLogitsLoss()


In [ ]:
for name, param in model.named_parameters():
    if "blocks.10" in name or "blocks.11" in name:
        param.requires_grad = True

optimizer = torch.optim.AdamW([
    {"params": model.head.parameters(), "lr": 1e-3},
    {"params": model.blocks[-2:].parameters(), "lr": 1e-5}
])


In [ ]:
from tqdm import tqdm

EPOCHS = 10

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0

    for imgs, labels in tqdm(loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        imgs = imgs.to(device)
        labels = labels.unsqueeze(1).to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1} Loss: {running_loss/len(loader):.4f}")

Epoch 1/10: 100%|██████████| 301/301 [02:25<00:00,  2.07it/s]


Epoch 1 Loss: 0.0604


Epoch 2/10: 100%|██████████| 301/301 [02:23<00:00,  2.10it/s]


Epoch 2 Loss: 0.0035


Epoch 3/10: 100%|██████████| 301/301 [02:23<00:00,  2.10it/s]


...

Epoch 8 Loss: 0.0002


Epoch 9/10: 100%|██████████| 301/301 [02:24<00:00,  2.09it/s]


Epoch 9 Loss: 0.0017


Epoch 10/10: 100%|██████████| 301/301 [02:23<00:00,  2.09it/s]

Epoch 10 Loss: 0.0007


In [ ]:
model.eval()

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
import cv2
import torch

infer_tfms = A.Compose([
    A.Resize(224,224),
    A.Normalize(),
    ToTensorV2()
])

In [ ]:
def predict_image(model, image_path, device):
    model.eval()

    # read image
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # preprocess
    img = infer_tfms(image=img)["image"]
    img = img.unsqueeze(0).to(device)

    with torch.no_grad():
        logits = model(img)
        prob = torch.sigmoid(logits).item()

    label = "AI-GENERATED (FAKE)" if prob > 0.5 else "REAL"
    confidence = prob if prob > 0.5 else 1 - prob

    return label, confidence

In [ ]:
label, confidence = predict_image(
    model,
    "/content/WhatsApp Image 2026-02-06 at 11.35.58 AM.jpeg",
    device
)

print("Prediction:", label)
print("Confidence:", round(confidence * 100, 2), "%")

Prediction: REAL
Confidence: 99.68 %


In [ ]:
SAVE_PATH = "/content/ai_face_detector_vit.pth"

torch.save(model.state_dict(), SAVE_PATH)
print(f"Model saved at: {SAVE_PATH}")

Model saved at: /content/ai_face_detector_vit.pth


In [ ]:
import os
import torch
import timm
import numpy as np
from PIL import Image
from tqdm import tqdm

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Using device: cuda


In [ ]:
class FaceDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.samples = []
        self.transform = transform

        ai_dir = os.path.join(root_dir, "AI-Generated Images")
        real_dir = os.path.join(root_dir, "Real Images")

        for img in os.listdir(ai_dir):
            self.samples.append((os.path.join(ai_dir, img), 1))  # AI = 1

        for img in os.listdir(real_dir):
            self.samples.append((os.path.join(real_dir, img), 0))  # Real = 0

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        img = Image.open(img_path).convert("RGB")

        if self.transform:
            img = self.transform(img)

        return img, torch.tensor(label, dtype=torch.float32)


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]
    )
])


In [ ]:
dataset_path = "/content/dataset/Human Faces Dataset"

dataset = FaceDataset(dataset_path, transform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=0)

print("Train samples:", len(train_ds))
print("Val samples:", len(val_ds))


Train samples: 7704
Val samples: 1926


In [ ]:
model = timm.create_model(
    "vit_base_patch16_224",
    pretrained=True,
    num_classes=1
).to(device)

# Ensure gradients are enabled
for p in model.parameters():
    p.requires_grad = True


In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=3e-4)

In [ ]:
EPOCHS = 5

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0
    correct = 0
    total = 0

    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):

        imgs = imgs.to(device)
        labels = labels.unsqueeze(1).to(device)

        optimizer.zero_grad()

        outputs = model(imgs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # -------- Accuracy --------
        probs = torch.sigmoid(outputs)
        preds = (probs > 0.5).float()
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct / total

    print(
        f"Epoch {epoch+1} | "
        f"Loss: {epoch_loss:.4f} | "
        f"Train Accuracy: {epoch_acc * 100:.2f}%"
    )


Epoch 1/5: 100%|██████████| 241/241 [04:05<00:00,  1.02s/it]


Epoch 1 | Loss: 0.1965 | Train Accuracy: 90.72%


Epoch 2/5: 100%|██████████| 241/241 [04:10<00:00,  1.04s/it]


Epoch 2 | Loss: 0.0123 | Train Accuracy: 99.60%


Epoch 3/5: 100%|██████████| 241/241 [04:07<00:00,  1.03s/it]


Epoch 3 | Loss: 0.0100 | Train Accuracy: 99.65%


Epoch 4/5: 100%|██████████| 241/241 [04:10<00:00,  1.04s/it]


Epoch 4 | Loss: 0.0074 | Train Accuracy: 99.79%


Epoch 5/5: 100%|██████████| 241/241 [04:12<00:00,  1.05s/it]

Epoch 5 | Loss: 0.0456 | Train Accuracy: 98.66%


In [ ]:
def evaluate_accuracy(model, loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(device)
            labels = labels.to(device)

            logits = model(imgs)
            probs = torch.sigmoid(logits).squeeze()
            preds = (probs > 0.5).float()

            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return correct / total


In [ ]:
val_acc = evaluate_accuracy(model, val_loader)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")

Validation Accuracy: 99.38%


In [ ]:
SAVE_PATH = "/content/ai_generated_face_detector_vit_best.pth"

torch.save(model.state_dict(), SAVE_PATH)

print(f"Model saved successfully at: {SAVE_PATH}")

Model saved successfully at: /content/ai_generated_face_detector_vit_best.pth


In [ ]:
from PIL import Image
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]
    )
])

In [ ]:
def predict_image(image_path, model, threshold=0.5):
    model.eval()

    img = Image.open(image_path).convert("RGB")
    img_tensor = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        logit = model(img_tensor)
        prob = torch.sigmoid(logit).item()

    label = "AI Generated" if prob > threshold else "Real"

    return label, prob

In [ ]:
image_path = "/content/real_face_photo.webp"
label, confidence = predict_image(image_path, model)

print("Prediction:", label)
print(f"Confidence: {confidence:.4f}")


Prediction: Real
Confidence: 0.3836


In [ ]:
import torch
import timm
from PIL import Image
from torchvision import transforms

device = "cuda" if torch.cuda.is_available() else "cpu"

MODEL_PATH = "/content/ai_face_detector_vit.pth"

model = timm.create_model(
    "vit_base_patch16_224",
    pretrained=False,
    num_classes=1
).to(device)

model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval()

print("✅ Model loaded")

✅ Model loaded


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]
    )
])


In [ ]:
def predict_image(image_path, model, threshold=0.5):
    img = Image.open(image_path).convert("RGB")
    img_tensor = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        logit = model(img_tensor)
        prob = torch.sigmoid(logit).item()

    label = "AI Generated (Fake)" if prob > threshold else "Real"

    return label, prob


In [ ]:
image_path = "/content/WhatsApp Image 2026-02-06 at 11.33.04 AM.jpeg"  # 👈 replace with your image path

label, confidence = predict_image(image_path, model)

print("Prediction:", label)
print(f"AI probability: {confidence:.4f}")


Prediction: AI Generated (Fake)
AI probability: 0.7379
